# __Datenbereinigung Teil 3: Bereinigung der Gesamtdaten__

In diesem Schritt werden die Gesamtdaten bereinigt.

In [16]:
from time import time
import pandas as pd 
from bs4 import BeautifulSoup
import string
import nltk
from nltk.corpus import stopwords  
from nltk.tokenize.treebank import TreebankWordDetokenizer
import os

# Working-Directory
os.chdir('C:/Users/Sven Konermann/Documents/Master/2_Semester/aF_E_Faelle/Unterlagen/00_all')

In [83]:
# Einlesen der Daten über ein csv File
data = pd.read_csv("Inserate_Gesamtdatenanalyse.csv", sep=";")
data.shape
#data = data.dropna()

print(data.head(5))
data.info()
#data.to_csv("Inserate_clean_Test.csv", sep=";")

   Unnamed: 0.1  Unnamed: 0           ad_id active_start_on active_end_on  \
0             6           6  jobsse-5759852      04.10.2014    05.05.2014   
1             7           7  jobsse-5879310      19.06.2014    14.07.2014   
2            12          12  jobsse-5700844      14.03.2014    26.03.2014   
3            13          13  jobsse-6271598      19.02.2015    27.02.2015   
4            15          15  jobsse-5634978      02.07.2014    02.11.2014   

                                               title  \
0                          IT-Projektleiter/in 100 %   
1  Projektleiter SAP Inkasso / Exkasso (FS-CD) (w/m)   
2                         ICT-Projektleiter (60-70%)   
3                        Projektleiter im SAP-Umfeld   
4                         IT- Projektleiter (Senior)   

                                                text  year  
0  IT-Projektleiter/in, Ref.-Nr. 39892D        Be...  2014  
1  <div id="jobTplContainer" style="overflow: hid...  2014  
2  <p><strong>Ref

In [84]:
# HTML-Elemente in einer neue Spalte 'Text_Anforderungen' abspeichern

start = time()

column = []
for i in data['text']:
    text=''
    soup = BeautifulSoup(i, 'lxml')
    text = soup.text 
    column.append(text)

data['Text_Anforderungen'] = column

print(data.Text_Anforderungen[0:5])
print('--------------------------')
print(data.shape)
print('--------------------------')
print(f'Benötigte Zeit: {time() - start} Sekunden')

0    IT-Projektleiter/in, Ref.-Nr. 39892D        Be...
1        Daniel Kapetanovic Leiter Rechnungswesen S...
2    Referenz Nr.:   4777SM    Branche:   BehÇôrden...
3    Referenz Nr.: 5045TG    Einsatzort: ZÇ¬rich   ...
4    Hauptaufgaben: - Management von grossen und ko...
Name: Text_Anforderungen, dtype: object
--------------------------
(41922, 9)
--------------------------
Benötigte Zeit: 37.52906155586243 Sekunden


In [85]:
# Lemmatisierung der Gesamtdaten (kann je nach Datenmenge länger dauern).
from HanTa import HanoverTagger as ht
start = time()
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

for mail in data['Text_Anforderungen']:
    lemma = [lemma for (word,lemma,pos) in tagger.tag_sent(mail.split())]
    index = data[data['Text_Anforderungen']==mail].index
    data.loc[index, 'Text_Anforderungen'] = (' '.join(lemma))
    
print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 9854.210126161575 Sekunden


In [86]:
# Text in Kleinbuchstaben umwandeln
data['Text_Anforderungen'] = (data['Text_Anforderungen'].str.lower())

In [87]:
# Satzzeichen löschen
def remove_punctuation(txt):
    txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunct

data['Text_Anforderungen'] = data['Text_Anforderungen'].apply(lambda x: remove_punctuation(x))

In [88]:
# \t-Tag entfernen aus den Texten
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("\t", "", regex=True)

In [89]:
# Manuell ähnliche Anforderungen zu einem Hauptbegriff ersetzen
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("teamplayer", "teamfähig", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("verhandlungssicher", "verhandlungsgeschick", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kundenorientierte", "kundenorientierung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("projekt management", "projektleitungserfahrung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("projektmanagement", "projektleitungserfahrung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("zusammenarbeit", "teamfähig", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kommunikation", "kommunikationsfähigkeit", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kommunikationsfähigkeiten", "kommunikationsfähigkeit", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("dwh", "data warehouse", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("dienstleistungsorientierung", "dienstleistungsorientiert", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("eigenverantwortung", "eigenverantwortlich", regex=True)

In [90]:
# Word-Tokenization 
start = time()
data['text_tok'] = data.apply(lambda row: nltk.word_tokenize(row['Text_Anforderungen']), axis=1)
print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 33.74079489707947 Sekunden


In [91]:
# Stopwörter löschen
stop_words = set(stopwords.words('german'))
data['text_tok'] = data['text_tok'].apply(lambda x: [item for item in x if item not in stop_words])

In [92]:
# Tokenizing rückgängig machen, da die Tokenizierung nicht in eine csv-Datei überführt werden kann
column = []
for i in data['text_tok']:
    test = TreebankWordDetokenizer().detokenize(i)
    column.append(test)

data['Text_Anforderungen'] = column

In [93]:
# Löschen der tokenisierten Spalte
data = data.drop(['text_tok'], axis = 1)

In [94]:
# Erneutes Löschen der Duplikate, da sich die Texte verändert haben
data = data.drop_duplicates(subset='Text_Anforderungen', keep="last")

In [95]:
data.to_csv("Inserate_clean_Gesamtdatenanalyse.csv", sep=";")

nächster Schritt: Mithilfe der erstellten Wörterbücher, die Gesamtdatenanalyse durchführen (Gesamtdatenanalyse.ipynb)